# Enterprise AI Memory with NeuroEmbed, NeuroIndex & Groq
### RAG • Chatbots • Incident Analysis • Conversational Memory


```
User Query
   ↓
NeuroEmbed (context-aware embedding)
   ↓
NeuroIndex (semantic memory)
   ↓
Retrieved Context
   ↓
Groq LLM
   ↓
Grounded Answer
```

### Why this matters

Most RAG systems solve only document Q&A.
This notebook demonstrates a unified semantic memory that supports:
- regulatory reasoning
- operational analysis
- conversational recall
- executive decision support

All using the same memory and embedding space.


### Limitations

- This notebook uses a small curated corpus for clarity.
- Performance gains from context increase with corpus size and ambiguity.
- In production, access control and metadata filtering should be added.


In [38]:
# pip install groq neuroembed neuroindex

What this notebook demonstrates

This notebook shows how a single semantic memory system can power:

- Regulatory RAG (finance / compliance)
- Incident Response Assistant
- Persistent Conversational Memory
- Decision Support & Synthesis

We use:
- NeuroEmbed → context-aware meaning
- NeuroIndex → persistent semantic memory
- Groq LLM → ultra-low-latency reasoning

In [ ]:
import os
from groq import Groq

from neuroindex import NeuroIndex
from neuroembed.core import NeuroEmbed
from neuroembed.encoders.sentence_transformer import SentenceTransformerEncoder

In [ ]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "gsk_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

client = Groq()

Initialize NeuroEmbed (Semantic Control)

In [ ]:
encoder = SentenceTransformerEncoder()

ne = NeuroEmbed(
    encoder=encoder,
    alpha=0.6
)

Initialize NeuroIndex (Enterprise Memory)

In [ ]:
ni = NeuroIndex(
    path="./enterprise_llm_memory",
    dim=112
)

Enterprise Knowledge Ingestion

In [ ]:
rbi_policy = """
The Reserve Bank of India regulates interest rates using monetary policy
tools such as the repo rate and reverse repo rate. Changes in these rates
directly affect bank lending, inflation control, and overall liquidity
in the financial system.
"""

In [ ]:
incident_report = """
On 14 June, a failure in the core banking system caused transaction
processing delays across multiple regions. The root cause was traced
to a database replication failure during a routine upgrade.
"""

In [ ]:
flood_report = """
Severe flooding near the river bank resulted in evacuation orders
for nearby villages. Authorities warned that rising water levels
could impact infrastructure and emergency services.
"""

Domain Contexts (Industry-Grade)

In [ ]:
finance_context = [
    "monetary policy",
    "central banking",
    "interest rates",
    "financial regulation"
]

ops_context = [
    "incident management",
    "banking infrastructure",
    "system outage",
    "root cause analysis"
]

risk_context = [
    "natural disaster",
    "flood risk",
    "infrastructure impact"
]

Store Knowledge in Semantic Memory

In [ ]:
def embed_ctx(text, ctx):
    return ne.embed(text, ctx).astype("float32")

ni.add_document("[FINANCE] RBI Monetary Policy\n" + rbi_policy,
                embed_ctx(rbi_policy, finance_context))

ni.add_document("[OPS] Core Banking Incident\n" + incident_report,
                embed_ctx(incident_report, ops_context))

ni.add_document("[RISK] Flood Impact Report\n" + flood_report,
                embed_ctx(flood_report, risk_context))

'4798db60ef98475a'

In [ ]:
mixed_doc = """
The bank announced temporary service shutdowns after flood warnings
raised concerns about ATM availability and branch safety.
"""

ni.add_document(
    "[MIXED] Bank Flood Operational Risk",
    embed_ctx(
        mixed_doc,
        finance_context + risk_context + ops_context
    )
)

'7321bffa7ac8a18f'

USE CASE 1: Regulatory RAG (Compliance Assistant)

In [ ]:
def retrieve_context(query, ctx, k=3):
    # 1. Embed query with context
    query_embedding = embed_ctx(query, ctx)

    # 2. Vector search
    results = ni.search(
        query_vector=query_embedding,
        k=k
    )

    # 3. Return retrieved text
    return "\n\n".join(r.text for r in results)

In [ ]:
def build_rag_prompt(query, context):
    return f"""
You are a senior financial compliance advisor.

Use ONLY the context below. If the answer is not present, say so.

Context:
{context}

Question:
{query}

Answer in a clear, professional tone.
"""

In [ ]:
def groq_chat(prompt):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content

In [ ]:
query = "How does RBI control bank lending rates?"

ctx = retrieve_context(query, finance_context)
prompt = build_rag_prompt(query, ctx)

answer = groq_chat(prompt)
print(answer)

The Reserve Bank of India (RBI) controls bank lending rates through the repo rate and reverse repo rate, which are monetary policy tools used to regulate interest rates. 

When the RBI lowers the repo rate, it becomes cheaper for banks to borrow money from the RBI, which in turn allows them to lend at lower interest rates to their customers. Conversely, when the RBI raises the repo rate, it becomes more expensive for banks to borrow money from the RBI, leading to higher lending rates for their customers.

Similarly, the reverse repo rate works in the opposite direction. When the RBI lowers the reverse repo rate, it becomes more expensive for banks to deposit excess funds with the RBI, encouraging them to lend more to their customers. When the RBI raises the reverse repo rate, it becomes cheaper for banks to deposit excess funds with the RBI, discouraging them from lending as much to their customers.

Therefore, by adjusting the repo and reverse repo rates, the RBI can influence bank le

USE CASE 2: Incident Response Assistant

In [ ]:
def incident_prompt(query, context):
    return f"""
You are a senior site reliability engineer.

Context:
{context}

Question:
{query}

Provide root cause and operational insight.
"""

In [ ]:
query = "Why were customer transactions delayed during the outage?"

ctx = retrieve_context(query, ops_context)
answer = groq_chat(incident_prompt(query, ctx))
print(answer)

**Incident Summary:**

On 14 June, a failure in the core banking system caused transaction processing delays across multiple regions. The root cause was traced to a database replication failure during a routine upgrade.

**Root Cause:**

The root cause of the incident was a database replication failure. This failure occurred during a routine upgrade of the core banking system. The upgrade process involved updating the database schema, which is a critical component of the system. However, the upgrade process was not properly tested, and it caused a replication failure between the primary and secondary databases.

**Operational Insight:**

The replication failure prevented the system from writing new transactions to the secondary database, causing a backlog of unprocessed transactions. As a result, the system was unable to process new transactions, leading to delays and disruptions in customer transactions.

**Key Factors Contributing to the Incident:**

1. **Insufficient Testing:** The 

USE CASE 3: Conversational Memory (Enterprise Chatbot)

In [ ]:
def store_chat(user, assistant):
    text = f"User: {user}\nAssistant: {assistant}"
    ni.add_document(
        "[CHAT] Conversation Turn",
        ne.embed(text, ["conversation"]).astype("float32")
    )

In [ ]:
def store_chat(user, assistant):
    text = f"User: {user}\nAssistant: {assistant}"
    ni.add_document(
        "[CHAT] Conversation Turn",
        ne.embed(text, ["conversation"]).astype("float32")
    )

In [ ]:
def recall_memory(query, k=3):
    # 1. Embed query in "conversation" semantic space
    query_embedding = ne.embed(
        query,
        ["conversation"]
    ).astype("float32")

    # 2. Vector search over semantic memory
    results = ni.search(
        query_vector=query_embedding,
        k=k
    )

    # 3. Return recalled conversation text
    return "\n\n".join(r.text for r in results)

In [ ]:
past = recall_memory("What did we discuss about RBI policy?")
print(past)

[FINANCE] RBI Monetary Policy

The Reserve Bank of India regulates interest rates using monetary policy
tools such as the repo rate and reverse repo rate. Changes in these rates
directly affect bank lending, inflation control, and overall liquidity
in the financial system.



USE CASE 4: Decision Support & Synthesis (Advanced)

In [ ]:
def decision_prompt(question, contexts):
    return f"""
You are an enterprise risk analyst.

Information:
{contexts}

Question:
{question}

Provide a concise decision-oriented summary.
"""

In [ ]:
# Finance lens
ctx = retrieve_context(
    "What risks do banks face due to flooding?",
    finance_context
)
print(groq_chat(decision_prompt("Explain the risk.", ctx)))

**Risk Identification:** Flood Impact on Bank Operations

**Risk Description:** Severe flooding near the river bank poses a significant operational risk to the bank, threatening its infrastructure, employees, and customers.

**Key Risks:**

1. **Infrastructure Disruption:** Flooding may damage or destroy bank branches, ATMs, and other critical infrastructure, disrupting banking services and operations.
2. **Employee and Customer Safety:** Evacuation orders and rising water levels may put employees and customers at risk, potentially leading to injuries or fatalities.
3. **Business Continuity:** Disruption to banking services may impact the bank's ability to operate, potentially leading to financial losses and reputational damage.
4. **Data Security:** Flooding may compromise the bank's data centers, putting sensitive customer information at risk of theft or loss.

**Recommendations:**

1. **Conduct a Risk Assessment:** Evaluate the potential impact of flooding on bank operations and dev

In [ ]:
# Operational lens
ctx = retrieve_context(
    "What operational issues arise during floods?",
    ops_context
)
print(groq_chat(decision_prompt("Explain the risk.", ctx)))

**Risk Summary:**

The risk is a potential operational disruption to the bank's services due to severe flooding near the river bank. The risk is characterized by:

- **Probability:** High (due to the severity of the flooding and evacuation orders)
- **Impact:** High (potential disruption to infrastructure and emergency services)
- **Exposure:** Moderate (bank's operations may be affected, but the extent is uncertain)

**Decision-Oriented Summary:**

**Recommendation:** Activate the bank's emergency response plan to ensure business continuity and minimize potential disruptions. Key actions include:

1. **Remote work arrangements:** Implement remote work for employees in affected areas to maintain business operations.
2. **Customer support:** Provide alternative channels for customer support and communication.
3. **Supply chain management:** Monitor and mitigate potential disruptions to critical supply chains.
4. **Risk monitoring:** Continuously monitor the situation and update the risk

In [ ]:
contexts = "\n\n".join([
    retrieve_context("interest rates", finance_context),
    retrieve_context("system outage", ops_context),
    retrieve_context("flood risk", risk_context)
])

answer = groq_chat(
    decision_prompt(
        "What risks should banks consider this quarter?",
        contexts
    )
)

print(answer)

**Risk Considerations for Banks this Quarter:**

Based on the provided information, banks should consider the following risks:

1. **Monetary Policy Risk**: Changes in RBI's repo and reverse repo rates may impact bank lending, inflation control, and liquidity, affecting overall financial stability.
2. **Operational Risk**: The recent core banking system failure highlights the need for robust IT infrastructure and disaster recovery plans to prevent transaction processing delays and reputational damage.
3. **Natural Disaster Risk**: Severe flooding near riverbanks poses a threat to infrastructure, emergency services, and business continuity, emphasizing the importance of disaster preparedness and risk mitigation strategies.

**Recommendations:**

* Monitor RBI's monetary policy decisions and adjust lending strategies accordingly.
* Conduct regular IT system audits and implement disaster recovery plans to prevent operational disruptions.
* Develop and implement flood risk management plans

- NeuroEmbed controls semantic meaning,
- NeuroIndex controls enterprise memory,
- Groq enables real-time reasoning at scale.